## Enhanced Sequence Inference Model
The model is based on [this paper](https://arxiv.org/abs/1609.06038).

### Import Related Packages

In [1]:
import os
import json
import zipfile
import time
import itertools

import numpy as np
import mxnet as mx
import multiprocessing as mp
import gluonnlp as nlp

from mxnet import gluon, nd, init
from mxnet.gluon import nn, rnn
from mxnet import autograd, gluon, nd
from d2l import try_gpu
import pandas as pd

# sklearn's metric function to evaluate the results of the experiment
from sklearn.metrics import accuracy_score, f1_score

# fixed random number seed
np.random.seed(9102)
mx.random.seed(9102)

## Data pipeline

### Load Dataset

See [dataloader](data_loader.ipynb) for how the training samples are obtained.

In [2]:
data_folder = 'data/imdb/'
file_name = 'train.csv'
file_path = data_folder + file_name
# dev: True - only use a small dataset
# create_vocab: True - create a new vocabulary from training data
dev = True
# load train file
if dev:
    # load only n rows
    nrows = 1000
    data = pd.read_csv(file_path, nrows=nrows)
else:
    # load as many as possible
    data = pd.read_csv(file_path)
data.shape

(1000, 4)

In [3]:
data.dropna(inplace=True)
data.shape, data.head()

((1000, 4),     movie_id                                          sentence1  \
 0  tt0097576  The story opens in Monument Valley, Utah, in 1...   
 1  tt2488496  Luke Skywalker has vanished. In his absence, t...   
 2  tt0407887  In voiceover, Irish-American mobster Frank Cos...   
 3  tt0209144  This is a complex story about Leonard Shelby (...   
 4  tt1285016  In October 2003, Harvard University student Ma...   
 
                                            sentence2  label  
 0  Chemistry between Ford and Connery puts this f...  False  
 1  You did it, J.J. ... You have ruined Star Wars...   True  
 2  No respect! This remake of "Internal Affairs",...  False  
 3  Scientific and psychological thriller. Who kne...   True  
 4  A Grossly Overrated But Still Very Good Film A...  False  )

In [4]:
label2num = {'contradiction':0, 'neutral':1, 'entailment':2}

In [20]:
# create a list of review a label paris.
dataset = [[left, right, label] for left, right, label in \
           zip(data['sentence1'], data['sentence2'], data['label']) if label!='-' ]
# randomly divide one percent from the training set as a verification set.
train_dataset, valid_dataset = nlp.data.train_valid_split(dataset, valid_ratio=.1)
len(train_dataset), len(valid_dataset)

(900, 100)

In [21]:
train_dataset[100]

icle catches Zero\'s eye. He rushes back to the hotel to show Gustave, that Madame D. has been found dead in her bathroom. A shocked Gustave immediately takes Zero on a train where they travel to Madame D.\'s estate.On the train, they stop by a barley field on a day that is known as the "Closing of the Frontier". Gustave and Zero see soldiers standing in the field. A group of soldiers enter and ask to see the documents of both men. Gustave shows his papers, but Zero has none as he is an immigrant. The main soldier orders Zero to come with him. Gustave defends Zero and gets into a fight with the soldiers. They pin the two of them against the wall, and Gustave, seeing Zero in trouble, barks at the soldiers, "TAKE YOUR HANDS OFF MY LOBBY BOY!"The fracas soon catches the attention of Inspector Henckels (Edward Norton), who enters the car, and quickly recognizes Gustave! When Gustave hears the names of Henckel\'s parents, he quickly remembers who they were, as well as Henckel\'s childhood n

### Data Processing

The purpose of the following code is to process the raw data so that the processed data can be used for model training and prediction. We will use the `SpacyTokenizer` to split the document into tokens, `ClipSequence` to crop the comments to the specified length, and build a vocabulary based on the word frequency of the training data. Then we attach the [Glove](https://nlp.stanford.edu/pubs/glove.pdf)[2]  pre-trained word vector to the vocabulary and convert each token into the corresponding word index in the vocabulary.
Finally get the standardized training data set and verification data set

In [22]:
# tokenizer takes as input a string and outputs a list of tokens.
tokenizer = nlp.data.SpacyTokenizer('en')
length_review = 300
length_plot = 600

from src.util import mp_tokenizer

tokenizer = mp_tokenizer(tokenizer, length_review, length_plot)

# Preprocess the dataset
train_dataset_token, train_data_lengths = tokenizer.process_dataset(train_dataset)
valid_dataset_token, valid_data_lengths = tokenizer.process_dataset(valid_dataset)

Done! Sequence clipping and get length Time=43.28s, #Sentences=900
Done! Sequence clipping and get length Time=38.23s, #Sentences=100


In [23]:
from src.util import mp_indexer
embedding = 'glove.42B.300d'
indexer = mp_indexer(train_dataset_token, embedding)
train_dataset = indexer.process_dataset(train_dataset_token)
valid_dataset = indexer.process_dataset(valid_dataset_token)

In [24]:
len(train_dataset)

900

### Bucketing and DataLoader
Since each sentence may have a different length, we need to use `Pad` to fill the sentences in a minibatch to equal lengths so that the data can be quickly tensored on the GPU. At the same time, we need to use `Stack` to stack the category tags of a batch of data. For convenience, we use `Tuple` to combine `Pad` and `Stack`.

In order to make the length of the sentence pad in each minibatch as small as possible, we should make the sentences with similar lengths in a batch as much as possible. In light of this, we consider constructing a sampler using `FixedBucketSampler`, which defines how the samples in a dataset will be iterated in a more economic way.

Finally, we use `DataLoader` to build a data loader for the training. dataset and validation dataset. The training dataset requires `FixedBucketSampler`, but the validation dataset doesn't require the sampler.

In [25]:
batch_size = 32
bucket_num = 10
bucket_ratio = 0.5


def get_dataloader():
    # Construct the DataLoader pad data, stack label and lengths
    batchify_fn = nlp.data.batchify.Tuple(nlp.data.batchify.Pad(axis=0), \
                                          nlp.data.batchify.Pad(axis=0),
                                          nlp.data.batchify.Stack())

    # n this example, we use a FixedBucketSampler,
    # which assigns each data sample to a fixed bucket based on its length.
    batch_sampler = nlp.data.sampler.FixedBucketSampler(
        train_data_lengths,
        batch_size=batch_size,
        num_buckets=bucket_num,
        ratio=bucket_ratio,
        shuffle=True)
    print(batch_sampler.stats())

    # train_dataloader
    train_dataloader = gluon.data.DataLoader(
        dataset=train_dataset,
        batch_sampler=batch_sampler,
        batchify_fn=batchify_fn)
    # valid_dataloader
    valid_dataloader = gluon.data.DataLoader(
        dataset=valid_dataset,
        batch_size=batch_size,
        shuffle=False,
        batchify_fn=batchify_fn)
    return train_dataloader, valid_dataloader

train_dataloader, valid_dataloader = get_dataloader()

FixedBucketSampler:
  sample_num=900, batch_num=36
  key=[57, 84, 111, 138, 165, 192, 219, 246, 273, 300]
  cnt=[7, 9, 8, 13, 6, 5, 14, 2, 1, 835]
  batch_size=[84, 57, 43, 34, 32, 32, 32, 32, 32, 32]


### Weighted softmax cross entropy loss
For a regular cross entropy loss, it is calculated by $-\vec{y}^\mathsf{T}\log \hat{\vec{y}}$. Since $\vec{y}$ is actually one-hot, so the value is $-\log\hat{y}_i$ where $i$ is the true label's index. To apply weight on this label, we multiply it with some weight value $w_i$, therefore, the loss of class $i$ is $-w_i\log\hat{y}_i$.

In [26]:
class WeightedSoftmaxCE(nn.HybridBlock):
    def __init__(self, sparse_label=True, from_logits=False,  **kwargs):
        super(WeightedSoftmaxCE, self).__init__(**kwargs)
        with self.name_scope():
            self.sparse_label = sparse_label
            self.from_logits = from_logits

    def hybrid_forward(self, F, pred, label, class_weight, depth=None):
        if self.sparse_label:
            label = F.reshape(label, shape=(-1, ))
            label = F.one_hot(label, depth)
        if not self.from_logits:
            pred = F.log_softmax(pred, -1)

        weight_label = F.broadcast_mul(label, class_weight)
        loss = -F.sum(pred * weight_label, axis=-1)
        
        return loss

In [0]:
class AlignAttention(nn.Block):
    def __init__(self, **kwargs):
        super(AlignAttention, self).__init__(**kwargs)
    
    def forward(self, inp_left, inp_right):
        # input dimension is of (batch_size, seq_len, embed_size)
        # att dimension is of (batch_size, seq_len_left, seq_len_right)
        att = nd.batch_dot(inp_left, nd.transpose(inp_right, axes = (0, 2, 1)))
        # inp_left_dot dimention is of (batch_size, seq_left, embed_size)
        inp_left_dot = nd.batch_dot(nd.softmax(att, axis=-1), inp_right)
        # inp_right_dot dimension is of (batch_size, seq_right, embed_size)
        inp_right_dot = nd.batch_dot(nd.softmax(nd.transpose(att, axes=(0, 2, 1)), axis=-1), inp_left)
        # concat original (lstm output, dot multiplier, substraction, elementwise product)
        # therefore, the real size is (batch_size, seq_len_left/right, embed_size*4)
        aug_left = nd.concat(inp_left, inp_left_dot, inp_left-inp_left_dot, inp_left*inp_left_dot, dim=-1)
        aug_right = nd.concat(inp_right, inp_right_dot, inp_right-inp_right_dot, inp_right*inp_right_dot, dim=-1)
        return aug_left, aug_right, att

In [27]:
class EnhancedSeqInfer(nn.Block):
    def __init__(self, vocab_len, embsize, nhidden, nlayers, nfc, nclass, drop_prob, **kwargs):
        super(EnhancedSeqInfer, self).__init__(**kwargs)
        with self.name_scope():
            # dropout prob
            self.drop_prob = drop_prob
            # word embedding
            self.embedding_layer = nn.Embedding(vocab_len, embsize)
            # first lstm, from sentence embed to hidden outputs
            self.bilstm1 = rnn.LSTM(nhidden, num_layers=nlayers, dropout=drop_prob, bidirectional=True)
            # second lstm, from augmented embed to m
            self.bilstm2 = rnn.LSTM(nhidden, num_layers=1, dropout=drop_prob, bidirectional=True)
            # enhancement
            self.align_att = AlignAttention()
            # this layer is used to output the final class
            self.output_layer = nn.HybridSequential()
            self.output_layer.add(nn.Dense(nfc, activation='tanh'), nn.Dropout(rate=drop_prob), \
                                  nn.Dense(nfc, activation='tanh'), nn.Dropout(rate=drop_prob), \
                                  nn.Dense(nclass))

    def forward(self, inp_left, inp_right):
        # inp is a list containing left_text and right_text
        # their size: [batch, token_idx]
        # inp_embed_left/right size: [batch, seq_len, embed_size]
        inp_embed_left = self.embedding_layer(inp_left)
        inp_embed_right = self.embedding_layer(inp_right)
        # rnn requires the first dimension to be the time steps, output is (seq_len, batch_size, embed_size)
        h_output_left = self.bilstm1(nd.transpose(inp_embed_left, axes=(1, 0, 2)))
        h_output_right = self.bilstm1(nd.transpose(inp_embed_right, axes=(1, 0, 2)))
        m_left, m_right, att = self.align_att(nd.transpose(h_output_left, axes=(1, 0, 2)), \
                                                      nd.transpose(h_output_right, axes=(1, 0, 2)))
        # apply another layer of lstm
        # v_left/right shape is (seq_len, batch_size, embed_size)
        v_left = self.bilstm2(nd.transpose(m_left, axes=(1, 0, 2)))
        v_right = self.bilstm2(nd.transpose(m_right, axes=(1, 0, 2)))
        # restore v's shape (batch_size, seq_len, embed_size)
        v_left = nd.transpose(v_left, axes=(1, 0, 2))
        v_right = nd.transpose(v_right, axes=(1, 0, 2))
        # apply max pooling 1D and avg pooling 1D
        v_left_avg = nd.sum(v_left, axis=1) / v_left.shape[1]
        v_right_avg = nd.sum(v_right, axis=1) / v_right.shape[1]
        v_left_max = nd.max(v_left, axis=1)
        v_right_max = nd.max(v_right, axis=1)
        # concatenate these 4 matrices
        dense_input = nd.concat(v_left_avg, v_left_max, v_right_avg, v_left_max, dim=-1)
        
        output = self.output_layer(dense_input)
        return output, att

### Configure parameters and build models
`test_model` is set to `True` if we want to just load parameters and test on the model. Otherwise, it is set to `False` during training.

In [28]:
test_model = False
vocab_len = len(indexer.vocab)
emsize = 300    # word embedding size
nhidden = 300   # lstm hidden_dim
nlayers = 2     # lstm layers

# final fc layer's number of hidden units and predicted number of classes
nfc = 256
nclass = 2

drop_prob = 0.5

ctx = mx.gpu(0)

model = EnhancedSeqInfer(vocab_len, emsize, nhidden, nlayers, nfc, nclass, drop_prob)

if test_model:
    model.load_parameters('model/esim-0.79.params', ctx=ctx)
else:
    model.initialize(init=init.Xavier(), ctx=ctx)
# Attach a pre-trained glove word vector to the embedding layer
model.embedding_layer.weight.set_data(indexer.vocab.embedding.idx_to_vec)
# fixed the embedding layer
model.embedding_layer.collect_params().setattr('grad_req', 'null')

print(model)

train_curve, valid_curve = [], []

EnhancedSeqInfer(
  (embedding_layer): Embedding(27841 -> 300, float32)
  (bilstm1): LSTM(None -> 300, TNC, num_layers=2, dropout=0.5, bidirectional)
  (bilstm2): LSTM(None -> 300, TNC, dropout=0.5, bidirectional)
  (align_att): AlignAttention(
  
  )
  (output_layer): HybridSequential(
    (0): Dense(None -> 256, Activation(tanh))
    (1): Dropout(p = 0.5, axes=())
    (2): Dense(None -> 256, Activation(tanh))
    (3): Dropout(p = 0.5, axes=())
    (4): Dense(None -> 2, linear)
  )
)


In [29]:
def calculate_loss(x_left, x_right, y, model, loss, class_weight):
    pred, att = model(x_left, x_right)
    y = nd.array(y.astype('int32', copy=False), ctx=ctx)
    if loss_name == 'sce':
        l = loss(pred, y)
    elif loss_name == 'wsce':
        l = loss(pred, y, class_weight, class_weight.shape[0])
    else:
        raise NotImplemented
    return pred, l

In [30]:
def one_epoch(data_iter, model, loss, trainer, ctx, is_train, epoch, class_weight=None, loss_name='sce'):

    loss_val = 0.
    total_pred = []
    total_true = []
    n_batch = 0

    for batch_x_left, batch_x_right, batch_y in data_iter:
        batch_x_left = batch_x_left.as_in_context(ctx)
        batch_x_right = batch_x_right.as_in_context(ctx)
        batch_y = batch_y.as_in_context(ctx)

        if is_train:
            with autograd.record():
                batch_pred, l = calculate_loss(batch_x_left, batch_x_right, \
                                               batch_y, model, loss, class_weight)

            # backward calculate
            l.backward()

            # update parmas
            trainer.step(batch_x_left.shape[0])

        else:
            batch_pred, l = calculate_loss(batch_x_left, batch_x_right, \
                                           batch_y, model, loss, class_weight)

        # keep result for metric
        batch_pred = nd.argmax(nd.softmax(batch_pred, axis=1), axis=1).asnumpy()
        batch_true = np.reshape(batch_y.asnumpy(), (-1, ))
        total_pred.extend(batch_pred.tolist())
        total_true.extend(batch_true.tolist())
        
        batch_loss = l.mean().asscalar()

        n_batch += 1
        loss_val += batch_loss

        # check the result of traing phase
        if is_train and n_batch % 400 == 0:
            print('epoch %d, batch %d, batch_train_loss %.4f, batch_train_acc %.3f' %
                  (epoch, n_batch, batch_loss, accuracy_score(batch_true, batch_pred)))

    # metric
    F1 = f1_score(np.array(total_true), np.array(total_pred), average='binary')
    acc = accuracy_score(np.array(total_true), np.array(total_pred))
    loss_val /= n_batch

    if is_train:
        print('epoch %d, learning_rate %.5f \n\t train_loss %.4f, acc_train %.3f, F1_train %.3f, ' %
              (epoch, trainer.learning_rate, loss_val, acc, F1))
        train_curve.append((acc, F1))
        # declay lr
        if epoch % 2 == 0:
            trainer.set_learning_rate(trainer.learning_rate * 0.9)
    else:
        print('\t valid_loss %.4f, acc_valid %.3f, F1_valid %.3f, ' % (loss_val, acc, F1))
        valid_curve.append((acc, F1))

In [31]:
def train_valid(data_iter_train, data_iter_valid, model, loss, trainer, \
                ctx, nepochs, class_weight=None, loss_name='sce'):

    for epoch in range(1, nepochs+1):
        start = time.time()
        # train
        is_train = True
        one_epoch(data_iter_train, model, loss, trainer, ctx, is_train, epoch, class_weight, loss_name)

        # valid
        is_train = False
        one_epoch(data_iter_valid, model, loss, trainer, ctx, is_train, epoch, class_weight, loss_name)
        end = time.time()
        print('time %.2f sec' % (end-start))
        print("*"*100)

### Train

In [32]:
class_weight = None
loss_name = 'sce'
optim_name = 'adam'
lr = 0.001
clip = 2.5
nepochs = 10

trainer = gluon.Trainer(model.collect_params(), optim_name, {'learning_rate': lr, 'clip_gradient': clip})

if loss_name == 'sce':
    loss = gluon.loss.SoftmaxCrossEntropyLoss()
elif loss_name == 'wsce':
    loss = WeightedSoftmaxCE()
    # the value of class_weight is obtained by counting data in advance. It can be seen as a hyperparameter.
    class_weight = nd.array([1., 3.], ctx=ctx)
else:
    print('loss function {} is not implemented!'.format(loss_name))
    raise NotImplemented

In [33]:
# train and valid
train_valid(train_dataloader, valid_dataloader, model, loss, \
            trainer, ctx, nepochs, class_weight=class_weight, loss_name=loss_name)

epoch 1, learning_rate 0.00100 
	 train_loss 0.6014, acc_train 0.716, F1_train 0.200,
	 valid_loss 0.6176, acc_valid 0.680, F1_valid 0.059,
time 26.95 sec
****************************************************************************************************
epoch 2, learning_rate 0.00100 
	 train_loss 0.5949, acc_train 0.718, F1_train 0.201,
	 valid_loss 0.5887, acc_valid 0.660, F1_valid 0.320,
time 26.78 sec
****************************************************************************************************


### Save the model and the training, validation curves

In [1]:
model.save_parameters('model/esim-{:.4}.params'.format(str(valid_curve[-1][0])))
with open('acc_record', 'w') as f:
    f.write(str(train_curve)+'\n')
    f.write(str(valid_curve))

NameError: name 'model' is not defined

### Visualize the training and validation curves
We can find in the plot that after how many epochs the model performs best on validation set without overfitting.

In [ ]:
from ast import literal_eval
with open('acc_record', 'r') as f:
    train_curve, valid_curve = [ literal_eval(line) for line in f]

In [ ]:
train_acc = [ acc for acc, _ in train_curve ]
train_f1 = [ f1 for _, f1 in train_curve ]
valid_acc = [ acc for acc, _ in valid_curve ]
valid_f1 = [ f1 for _, f1 in valid_curve ]
epochs = [ i for i in range(1, len(valid_curve)+1) ]
import matplotlib.pyplot as plt
plt.title('Accuracy')
plt.plot(epochs, train_acc, label='train', color='orange')
plt.plot(epochs, valid_acc, label='validation', color='green')
plt.legend(loc='upper left')
plt.show()
plt.title('F1')
plt.plot(epochs, train_f1, label='train', color='orange')
plt.plot(epochs, valid_f1, label='validation', color='green')
plt.legend(loc='upper left')
plt.show()

### Test on randomly made up plot and review by myself
Now we will randomly input a movie plot summary and its review to see if the model decide the review is a spoiler or not.

In [ ]:
right = 'Tim killed Tom on a winter evening, and Sarah saw this scene.'
left = 'Sarah saw Tom killed, it is Tim to blame'
left_token = indexer.vocab[tokenizer.tokenizer(left)]
right_token = indexer.vocab[tokenizer.tokenizer(right)]

In [ ]:
left_input = nd.array(left_token, ctx=ctx).reshape(1,-1)
right_input = nd.array(right_token, ctx=ctx).reshape(1,-1)
pred, att = model(left_input, right_input)
pred, nd.argmax(pred, axis=1).asscalar()

### Visualize the soft alignment

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
vis_att = np.squeeze(att.asnumpy())
plt.figure(figsize=vis_att.T.shape)
sns.heatmap(vis_att, xticklabels=tokenizer.tokenizer(right), yticklabels=tokenizer.tokenizer(left), center=0.)